# 3_training_mlflow.ipynb

- **Objectif** :
Lancer et suivre les entraînements des modèles ML (Logistic, Random Forest, XGBoost)
avec différentes stratégies de déséquilibre, via MLflow.
---

In [94]:
import os
import mlflow
import mlflow.sklearn
from IPython.display import Markdown as md

## Définir le dossier du projet

In [95]:
# Définir le dossier du projet
os.chdir("..") if "notebooks" in os.getcwd() else None

## Configurer le tracking local MLflow

In [96]:
# Configurer le tracking local MLflow
mlflow.set_tracking_uri("file:./mlruns")
mlflow.set_experiment("Churn_Prediction_Models")


<Experiment: artifact_location='file:///c:/Users/PC/Desktop/sprint5/Churn-Prediction-avec-MLflow/mlruns/667014208243130233', creation_time=1761207273202, experiment_id='667014208243130233', last_update_time=1761207273202, lifecycle_stage='active', name='Churn_Prediction_Models', tags={'mlflow.experimentKind': 'custom_model_development'}>

In [97]:
md("## 🚀 Lancement des entraînements et suivi MLflow")

## 🚀 Lancement des entraînements et suivi MLflow

## 1️⃣ Lancer les scripts

In [98]:
# ==========================================================
# 1️⃣ Lancer les scripts
# ==========================================================
import subprocess

In [99]:
scripts = [
    "scripts/logistic_script.py",
    "scripts/forest_script.py",
    "scripts/xgboost_script.py"
]

In [100]:
for script in scripts:
    print(f"⚙️ Entraînement avec {script} ...")
    subprocess.run(["python", script])
    print(f"✅ Terminé : {script}")

⚙️ Entraînement avec scripts/logistic_script.py ...
✅ Terminé : scripts/logistic_script.py
⚙️ Entraînement avec scripts/forest_script.py ...
✅ Terminé : scripts/forest_script.py
⚙️ Entraînement avec scripts/xgboost_script.py ...
✅ Terminé : scripts/xgboost_script.py


In [101]:
import mlflow

# Récupérer l'ID de l'expérience
experiment = mlflow.get_experiment_by_name("Churn_Prediction_Models")
experiment_id = experiment.experiment_id

# Chercher tous les runs de l'expérience
runs = mlflow.search_runs(experiment_ids=[experiment_id], filter_string="", order_by=["start_time DESC"])
print(runs.columns.tolist())


['run_id', 'experiment_id', 'status', 'artifact_uri', 'start_time', 'end_time']


## 2️⃣ Comparaison des résultats

In [102]:
# ==========================================================
# 2️⃣ Comparaison des résultats
# ==========================================================
md("### 📊 Visualisation dans MLflow UI")

### 📊 Visualisation dans MLflow UI

In [103]:
print("""
Pour ouvrir l'interface MLflow :
> mlflow ui --backend-store-uri ./mlruns
Puis ouvrir : http://127.0.0.1:5000
""")


Pour ouvrir l'interface MLflow :
> mlflow ui --backend-store-uri ./mlruns
Puis ouvrir : http://127.0.0.1:5000



In [104]:
md("### 🏆 Sélection du meilleur modèle")

### 🏆 Sélection du meilleur modèle

In [105]:
import pandas as pd

## Charger les runs MLflow

In [106]:
# Charger les runs MLflow
client = mlflow.tracking.MlflowClient()
experiment = client.get_experiment_by_name("Churn_Prediction_Models")
runs = mlflow.search_runs(experiment_ids=[experiment.experiment_id])

In [107]:
# Vérifier les colonnes disponibles
print("📋 Colonnes disponibles :", runs.columns.tolist())

# Rechercher la colonne correspondant à la métrique F1
f1_cols = [col for col in runs.columns if "f1" in col.lower() and col.startswith("metrics.")]

# Si aucune métrique F1 n'est trouvée, afficher un message explicatif au lieu de lever une erreur
if not f1_cols:
    print("⚠️ Aucune métrique F1 trouvée dans les runs MLflow.")
    print("👉 Vérifiez que la métrique F1 a bien été loggée avec `mlflow.log_metric('f1', valeur)`.")
    print("🔎 Exemple de colonnes disponibles liées aux métriques :",
          [col for col in runs.columns if col.startswith("metrics.")])
else:
    # Sélectionner la première colonne F1 trouvée
    f1_col = f1_cols[0]

    # Trier les runs selon la meilleure valeur de F1
    best_run = runs.sort_values(f1_col, ascending=False).iloc[0]

    # Afficher le meilleur modèle avec sa valeur F1
    md(f"**🏆 Meilleur modèle :** {best_run.get('tags.model_name', 'Nom inconnu')} "
       f"({f1_col} = {best_run[f1_col]:.3f})")


📋 Colonnes disponibles : ['run_id', 'experiment_id', 'status', 'artifact_uri', 'start_time', 'end_time']
⚠️ Aucune métrique F1 trouvée dans les runs MLflow.
👉 Vérifiez que la métrique F1 a bien été loggée avec `mlflow.log_metric('f1', valeur)`.
🔎 Exemple de colonnes disponibles liées aux métriques : []


In [171]:
runs.filter(like="metrics.")


,metrics.accuracy,metrics.f1
0,0.5,0.5


In [172]:
best_run = runs.sort_values("metrics.f1", ascending=False).iloc[0]
md(f"**Meilleur modèle :** {best_run['tags.model_name']} (F1 = {best_run['metrics.f1']:.3f})")


**Meilleur modèle :** DummyModel (F1 = 0.500)

## Chargement et test du modèle

In [173]:
# Chargement et test du modèle
model_uri = f"runs:/{best_run['run_id']}/model"
loaded_model = mlflow.sklearn.load_model(model_uri)

In [174]:
md("### 🔍 Test d'une prédiction avec le modèle chargé")
import joblib
import pandas as pd

## Exemple d’un batch de test

In [ ]:
# Exemple d’un batch de test
X_test = joblib.load("data/X_test.pkl")
print("✅ Prédiction exemple :")
print(loaded_model.predict(X_test[:5]))

✅ Prédiction exemple :
[0 0 0 0 0]
